TODO: 
    
    - separar en carga raw, manipulacion, eda
    - no hacer ese fill na cuando paso bool a int (astype?)
    - sacar las listas temporales de cols a elegir y quedarme con cols
    - checkear el astype int .... tema bools: creo que deberia crear la de oficial store y dejarlas como tipo object

In [39]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import joblib 

In [2]:
os.chdir("..") 

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None


In [4]:
data_path = "data/"

In [5]:
input_path = data_path + 'input/'

# 1️⃣ Dataload 
---

In [6]:
url_base = "https://api.mercadolibre.com/"

In [7]:
url_MLA = url_base + "/sites/MLA/"

In [8]:
limit = 50 # fixed

In [9]:
items_per_category = 1000 # chosen

In [10]:
requests_per_category = int(items_per_category / limit)
requests_per_category

20

## Fxs

In [11]:
def get_data(url):
    # makes a get request given a url, and returns json response
    response = urllib.request.urlopen(url)
    return json.load(response)
    

In [12]:
def load_category_items(categ_id):
    # gets items given a category id
    url_categ = url_MLA + "search?&category={}".format(categ_id)
    all_results = [] # offset and limit must be taken into account to reach items_per_category
    for r in range(0, requests_per_category):
        offset = r * limit
        url = url_categ + "&offset={}".format(offset)
        response = get_data(url)
        results = response['results'] # keep only the items
        all_results = all_results + results
    return all_results 
    

## Get root categories

In [13]:
url = url_MLA + "categories"

In [14]:
root_categories = get_data(url)
root_categories

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas y Construcción'},
 {'id': 'MLA1574', 'name': 'Hogar, Muebles y Jardín'},
 {'

## Get items per root category

In [15]:
df = pd.DataFrame() # will contain items from all root categories
for c in tqdm(root_categories):
    all_results = load_category_items(c["id"])
    df_tmp = pd.json_normalize(all_results) # build a pd df from a json with nested data (notice how lists remain lists)
    # add root category data to the df with category items
    df_tmp["root_category_id"] = c["id"]
    df_tmp["root_category_name"] = c["name"]
    # append df_tmp to the final df
    df = df.append(df_tmp, ignore_index=True)

  0%|          | 0/31 [00:00<?, ?it/s]

**Inspect df**

In [16]:
df.shape

(30907, 132)

In [17]:
df.columns.tolist()

['id',
 'site_id',
 'title',
 'price',
 'sale_price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'stop_time',
 'condition',
 'permalink',
 'thumbnail',
 'thumbnail_id',
 'accepts_mercadopago',
 'attributes',
 'original_price',
 'category_id',
 'official_store_id',
 'domain_id',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'use_thumbnail_id',
 'order_backend',
 'seller.id',
 'seller.permalink',
 'seller.registration_date',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.eshop.nick_name',
 'seller.eshop.eshop_rubro',
 'seller.eshop.eshop_id',
 'seller.eshop.eshop_locations',
 'seller.eshop.site_id',
 'seller.eshop.eshop_logo_url',
 'seller.eshop.eshop_status_id',
 'seller.eshop.seller',
 'seller.eshop.eshop_experience',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transacti

In [18]:
df.columns[df.isna().any()].tolist()

['price',
 'sale_price',
 'currency_id',
 'condition',
 'thumbnail_id',
 'original_price',
 'official_store_id',
 'domain_id',
 'catalog_product_id',
 'catalog_listing',
 'seller.permalink',
 'seller.registration_date',
 'seller.tags',
 'seller.eshop.nick_name',
 'seller.eshop.eshop_rubro',
 'seller.eshop.eshop_id',
 'seller.eshop.eshop_locations',
 'seller.eshop.site_id',
 'seller.eshop.eshop_logo_url',
 'seller.eshop.eshop_status_id',
 'seller.eshop.seller',
 'seller.eshop.eshop_experience',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.

Search for specific columns 

In [19]:
[x for x in df.columns if "installments" in x]

['installments.quantity',
 'installments.amount',
 'installments.rate',
 'installments.currency_id',
 'installments']

In [20]:
df['installments.rate'].value_counts()

70.17    26021
0.00      2640
Name: installments.rate, dtype: int64

In [21]:
df.category_id[0]

'MLA6867'

## Get subcategories info
Recursive algorithm

In [22]:
def get_subcategories_from_root(categ_id, dic):
    # recursive function to get subcategories data 
    url = url_base + "categories/{}".format(categ_id)
    response = get_data(url)
    children_categories = response["children_categories"]
    for cc in children_categories: # base case: children_categories list is empty
        id = cc["id"]
        dic[id] = {"name": cc["name"], 
                    "root": response["path_from_root"]} # 'root' does not include current category as response["path_from_root"] has parent category data 
        get_subcategories_from_root(id, dic)  

**Get subcategories info for all root categories**

In [23]:
dic_all_categories = {}
for c in tqdm(root_categories):
    dic = {}
    get_subcategories_from_root(c["id"], dic)  
    dic_all_categories.update(dic)

  0%|          | 0/31 [00:00<?, ?it/s]

**Inspect dic_all_categories**

In [24]:
len(dic_all_categories)

9541

In [25]:
first_key = next(iter(dic_all_categories)) 
first_key

'MLA4711'

In [26]:
dic_all_categories[first_key]

{'name': 'Acc. para Motos y Cuatriciclos',
 'root': [{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'}]}

**Persist dic_all_categories**

In [42]:
file_name = input_path + "dic_all_categories.joblib"
joblib.dump(dic_all_categories, file_name)

['data/input/dic_all_categories.joblib']

# 2️⃣ Manipulación básica
---

In [29]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,None,ARS,1,22,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'source': 1, 'name': 'Marca del stereo', 'va...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[brand_verified, extended_warranty_eligible, g...",True,True,1,8911722,http://perfil.mercadolibre.com.ar/AR+ESCALADA,2011-04-19

## Add columns

**Add discount column**

In [30]:
df["discount"] = 1 - df.price / df.original_price if df.original_price is not None else None

In [31]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,None,ARS,1,22,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'source': 1, 'name': 'Marca del stereo', 'va...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[brand_verified, extended_warranty_eligible, g...",True,True,1,8911722,http://perfil.mercadolibre.com.ar/AR+ESCALADA,2

Inspect discount coulumn

In [32]:
df[~df.original_price.isna()][["id", "site_id", "title", "price", "sale_price", "original_price", "discount"]]

,id,site_id,title,price,sale_price,original_price,discount
3,MLA836983125,MLA,Funda Cubre Rueda Auxilio Ford Ecosport,891.00,None,990.0,0.1
7,MLA835929721,MLA,Soporte Respaldo Apoyo Lumbar Masajeador Asien...,700.00,None,1250.0,0.44
13,MLA836882881,MLA,Mini Compresor De Aire Portatil 12v 250 Psi Kl...,1003.00,None,1249.0,0.196958
14,MLA785918507,MLA,Kit Led Cree Cob H1 H7 H3 9005 12/24v 2019 C6+...,999.00,None,1600.0,0.375625
16,MLA836881964,MLA,Compresor De Aire Alto Impacto Portátil 250 Ps...,1599.00,None,1909.0,0.162389
...,...,...,...,...,...,...,...
30871,MLA905514102,MLA,Rack Mineria Rig Minero 6 Gpus Aluminio,7599.05,None,7999.0,0.05
30873,MLA875687395,MLA,Bandeja Lion Rolling Circus 27x16 Cm Salamanca...,939.06,None,999.0,0.06
30882,MLA868232362,MLA,"Pack X 4 Sahumerios Naturales Copal, Incienso,...",949.05,None,999.0,0.05
30885,MLA846077762,MLA,Lote De 40 Escallas En Piedras Semipreciosas A...,722.00,None,760.0,0.05


**Add category name (specific one for each item)**

In [33]:
df["category_name"] = df.category_id.apply( lambda x: dic_all_categories[x]["name"] if x in dic_all_categories.keys() else None )

In [34]:
def build_categories_from_root(lis):
    res = []
    for x in lis: 
        res.append(x["name"])
    return res

In [35]:
df["categories_from_root"] = df.category_id.apply( lambda x: build_categories_from_root(dic_all_categories[x]["root"]) + [df.category_name] if x in dic_all_categories.keys() else None )

In [36]:
df.head()

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount,category_name,categories_from_root
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,None,ARS,1,22,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'source': 1, 'name': 'Marca del stereo', 'va...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"[brand_verified, extended_warranty_eligible, g...",True,True,1,8911722,http://perfi

## Persist "raw" data

In [43]:
file_name = input_path + "raw_data.csv"
df.to_csv(file_name, sep='\t', encoding='utf-8', index=False)